In [ ]:
# Deployment application
import torch
import torch_tensorrt
import torch.backends.cudnn as cudnn
import time
import numpy as np

In [ ]:
model = torch.jit.load("tensorrt_module.ts")
input_data = torch.randn(1, 3, 224, 224)
input_data = input_data.to("cuda").float()
result = model(input_data)

In [ ]:
cudnn.benchmark = True

def benchmark(model, input_shape=(1024, 3, 512, 512), dtype='fp32', nwarmup=50, nruns=1000):
    input_data = torch.randn(input_shape)
    input_data = input_data.to("cuda")
    if dtype=='fp16':
        input_data = input_data.half()
        
    print("Warm up ...")
    with torch.no_grad():
        for _ in range(nwarmup):
            features = model(input_data)
    torch.cuda.synchronize()
    print("Start timing ...")
    timings = []
    with torch.no_grad():
        for i in range(1, nruns+1):
            start_time = time.time()
            pred_loc  = model(input_data)
            torch.cuda.synchronize()
            end_time = time.time()
            timings.append(end_time - start_time)
            if i%10==0:
                print('Iteration %d/%d, avg batch time %.2f ms'%(i, nruns, np.mean(timings)*1000))

    print("Input shape:", input_data.size())
    print('Average throughput: %.2f images/second'%(input_shape[0]/np.mean(timings)))

In [ ]:
model = model.eval().to("cuda")
benchmark(model, input_shape=(1, 3, 224, 224), nruns=100)

In [ ]:
results = model(torch.randn(1,3,224,224).float().to('cuda'))